In [ ]:

######## setup variables ##########

#gs_prefix="gs://anand-31-simple-files/aml-customer-extended"
gs_prefix = "gs://anand-31-simple-files/partition-test/"
partitions = 3
platform_prefix=gs_prefix

print("--")


In [ ]:
import os                                                                        
import time                                                                      
import json
import pandas as pd
from timeit import default_timer as timer
from katana.remote import aio, analytics, import_data, trace

from katana import remote
from katana.remote import import_data
from katana.remote import PartitionPolicy

# os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"
os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

print("--")


In [ ]:
my_client=remote.Client()
my_client.server_version

In [ ]:

my_graph = my_client.create_graph(
    num_partitions=partitions,
    name="partition_test"
)

print("--")


In [ ]:
# ######################################################

In [ ]:

#  CONNECT TO GRAPH

DB_NAME    = "default"
GRAPH_NAME = "partition_test"

# DB_NAME    = "my_db"
# GRAPH_NAME = "my_graph"


for l_graph in rc.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "     DB GRAPH: ", l_graph.name, "    GRAPH VERSION: ", l_graph.version)
      #
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)

        
print(my_graph.name)
print("--")


In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


In [ ]:
# ######################################################

In [ ]:
test_node_file = platform_prefix + "nodes.txt"
test_edge_file = platform_prefix + "edges.txt"
input_path=platform_prefix 

print("--")


In [ ]:
start = timer()
print("Importing graph from CSV files...")
import_data.csv(
    my_graph,
    input_node_path=test_node_file,
    input_edge_path=test_edge_file,
    input_dir=input_path,
    partition_policy=PartitionPolicy.RANDOM_OEC,
#    have_headers=True,
    data_delimiter=",",
    schema_delimiter=","
)
end = timer()
print(f"  import: {end - start:.1f} seconds") 

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
query = """
    MATCH(c:CUSTOMER)-[:LIVES_IN]-(a:ADDRESS)
    where c.ID = 'c_0'
    return c.ID, a.ID
"""
result = my_graph.query(query)
result.view()

# Import ends here: DATA LOADING COMPLETE

In [12]:

def visit(l_graph, nodeLabel, listOfAttributes):
    
    from katana.local_native import Node, OutEdgeHandle
    from katana_enterprise.distributed import single_host
    
    # output=[]
    
    id_property = l_graph.get_node_property('ID')
    # id_property = graph.get_node_property('id')
    
    with open('/tmp/' + 'AAA.txt', 'w') as f:
        
        for node in l_graph.master_nodes():
            # output.append(str(id_property[node]))
            
            # f.write(str(id_property[node])+ '#\n')
            f.write("MM2 - " + str(id_property[node])+ "\n")
            
        # for node in l_graph.mirror_nodes():
        #     f.write(str(id_property[node])+ '#m#\n')
            
    return single_host('')


# graph.run(lambda g: dist(g))

out = my_graph.run_result(lambda g: visit(g, 'PERSON', ['name']))



          0/? [?op/s]

In [ ]:
print(out)

In [ ]:
# out.value

In [ ]:

def write_number_of_addresses_feature(graph, nodeLabel, listOfAttributes):
    
    from katana.local_native import Node, OutEdgeHandle
    from katana_enterprise.distributed import single_host
    
    id_property = graph.get_node_property('ID')
    
    with open('/tmp/' + 'random_oec_customer_feature_number_of_address.txt', 'w') as f:
        for node in graph.master_nodes():
            print('do nothing')
            edges = graph.out_edge_ids(Node(node))
            address_count = 0
            for edge_id in edges:
                destination_node = graph.out_edge_dst(edge_id).value
                if str(id_property[destination_node]).startswith('a_'):
                    address_count = address_count + 1
            if str(id_property[node]).startswith('c_'):
                f.write( str(id_property[node]) + ',' + str(address_count) +'\n' )
                # f.write(str(id_property[node]) + ' has too many addresses:'+ str(len(edges))+'\n')
                
    return single_host('ok')

# graph.run(lambda g: dist(g))

out = graph.run_result(lambda g: write_number_of_addresses_feature(g, 'PERSON', ['name']))




In [ ]:
out.value